In [2]:
import alpaca_trade_api as tradeapi
import config
import logging
from alpaca_trade_api.rest import REST, TimeFrame
import datetime as dt
import pandas as pd
from datetime import date
import datetime
import yfinance as yf


#ally api endpoint creation
api = tradeapi.REST(key_id=config.LIVE_API_KEY, secret_key=config.LIVE_SECRET_KEY, api_version='v2',
                                        base_url=config.LIVE_API_BASE_URL)

positions = api.list_positions() # get account positions


#set start and end dates automatically
now = date.today()
today = now.strftime('%Y-%m-%d')
yesterday = (now - pd.Timedelta('1day')).strftime('%Y-%m-%d')
ninedaysago = (now - pd.Timedelta('8W')).strftime('%Y-%m-%d')





Symbols = input("Enter stock tickers separated by a comma: \n").split(', ')
tickers = yf.Tickers(','.join(Symbols))
for symbol in Symbols:
    print(tickers.tickers[symbol.upper()].get_news)

#calculate percentage change of a stock's prrice between two periods in time. In this case,
#this prints the 2 month change of a given stock ticker.

def percent_change():
    quote_iter = api.get_bars(i, TimeFrame.Day, start = ninedaysago, end = yesterday, limit=9)._raw
    quote_df = pd.DataFrame(quote_iter)
    ninedago = quote_df.iloc[0]['c']
    pricenow = quote_df.iloc[8]['c']
    percent_change = (pricenow - ninedago) / ninedago * 100
    print(i, ' moved {:.2f}% over the last 2 months'.format(percent_change))
   


bool_long = bool
bool_short = bool
short_df = pd.DataFrame()
long_df = pd.DataFrame()


# create dataframe adding EMA columns to date, stock,     




def dataframe_creator():
    target_symbols_long = []
    target_symbols_short = []
    quote_iter = api.get_bars(i, TimeFrame.Day, start = ninedaysago, end = yesterday, limit=80)._raw
    quotes_df0 = pd.DataFrame(quote_iter)
    quotes_df = pd.DataFrame(index=None)
    quotes_df['Date'] = pd.to_datetime(quotes_df0['t'])
    quotes_df['Date'] = quotes_df['Date'].dt.strftime('%m/%d/%Y')
    quotes_df['Symbol'] = i
    quotes_df['Close'] = quotes_df0['c']
    quotes_df["EMA9"] = quotes_df0['c'].ewm(span=9).mean().round(2)
    quotes_df["EMA20"] = quotes_df0['c'].ewm(span=20).mean().round(2)
    quotes_df["EMA50"] =quotes_df0['c'].ewm(span=50).mean().round(2)
    quotes_df['Volume'] = quotes_df0['v'].apply(lambda x : "{:,}".format(x))
    bool_long = quotes_df["EMA9"].iloc[-1] < quotes_df["Close"].iloc[-1]  ##create Boolean to filter out stocks to go long on.
    if bool_long == True:
        target_list_long = i
        target_symbols_long.append(target_list_long)
        print("Stock ", i, "is within buying parameters. Dataframe: " , "\n")
        print(quotes_df.tail(1))
    else:
        print("Stock", i, "is not within buying parameters")

    bool_short = quotes_df["EMA9"].iloc[-1] > quotes_df["Close"].iloc[-1]  ##create second boolean to filter out stocks to short.
    if bool_short == True:
        target_list_short = i
        target_symbols_short.append(target_list_short)
        print("Stock ", i, "is within selling parameters. Dataframe: " , "\n")
        print(quotes_df.tail(1))
    else:
        print("Stock", i, "is not within selling parameters")
    
    
#more work needs to be done on slicing into the optimal option chain rows. 
#Example: for stock i find options where price is <.5. How to handle multiple results?


    
def option_chain_generator():
    chain_symbol = yf.Ticker(str(i))
    chain = chain_symbol.option_chain(str("2023-01-20"))
    if bool_long == True:
        try:
            optc = chain_symbol.option_chain("2023-01-20")
            optcz = optc.calls[['strike', 'lastPrice', 'bid', 'ask', 'volume', 'openInterest', 'impliedVolatility']][2:16]
            optczdf = pd.DataFrame(optcz)
            long_df.join(optczdf) 
            optczdf['lastTradeDate'] = optczdf['lastTradeDate'].dt.strftime('%m/%d/%Y')
            optczdf.rename(columns = {'lastTradeDate':'lstTrade', 'lastPrice': 'last', 'openInterest': 'oInterest', 'impliedVolatility': 'iVol'}, inplace=True)
            print("\n", "BUY", i, ", Optimal Call Chain:", "\n\n", optczdf, "\n\n")
        except ValueError as err:
            print("no calls available for ", i, "\n\n")

    else:
        try:
            optp = chain_symbol.option_chain("2023-01-20")
            optpz = optp.puts[['lastTradeDate', 'strike', 'lastPrice', 'bid', 'ask', 'volume', 'openInterest', 'impliedVolatility']][20:50]
            optpzdf = pd.DataFrame(optpz)
            short_df.join(optpzdf)
            optpzdf['lastTradeDate'] = optpzdf['lastTradeDate'].dt.strftime('%m/%d/%Y')
            optpzdf.rename(columns = {'lastTradeDate':'lstTrade', 'lastPrice': 'last', 'openInterest': 'oInterest', 'impliedVolatility': 'iVol'}, inplace=True)
            print("\n", "SELL", i, ", Optimal Put Chain:", "\n\n", optpzdf, "\n\n")
        except ValueError as err:
            print("no puts available for ", i, "\n\n")    

    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
#dataframe_creator()
for i in Symbols:
    option_chain_generator()






    

Enter stock tickers separated by a comma: 
JPM, F, TSLA
<bound method TickerBase.get_news of yfinance.Ticker object <JPM>>
<bound method TickerBase.get_news of yfinance.Ticker object <F>>
<bound method TickerBase.get_news of yfinance.Ticker object <TSLA>>

 SELL JPM , Optimal Put Chain: 

       lstTrade  strike   last    bid    ask  volume  oInterest      iVol
20  01/05/2023   123.0   0.57   0.51   0.55    2334        935  0.375495
21  01/05/2023   124.0   0.63   0.60   0.64       7        731  0.368659
22  01/05/2023   125.0   0.74   0.71   0.74    1571      26066  0.360846
23  01/05/2023   126.0   0.90   0.83   0.87      34        353  0.355475
24  01/05/2023   127.0   0.94   0.98   1.02       9       1191  0.349860
25  01/05/2023   128.0   1.22   1.15   1.19     107       2745  0.344001
26  01/04/2023   129.0   1.45   1.33   1.39       1        277  0.338630
27  01/05/2023   130.0   1.59   1.57   1.64     157      18929  0.335700
28  01/05/2023   131.0   1.92   1.82   1.89      46 

# 

In [3]:
tsla.options

NameError: name 'tsla' is not defined